In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization, Conv2D, UpSampling2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, precision_recall_curve, accuracy_score, zero_one_loss, balanced_accuracy_score, average_precision_score
from inspect import signature
import matplotlib.pyplot as plt
import numpy as np
import tensorflow
import os
import sys
import logging
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras import backend

In [51]:
backend.set_image_data_format('channels_first')

In [52]:
#Sacar los mensajes de debugging de tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)
'''
    Con este se obtuvo un alto accuracy (mas del 90% para la estacion Cerro Obero oversampleada).
    Para correrlo, asegurarse que X_data_dir corresponde a una X con 3 alturas y con los -1 eliminados de X y de Y, 
    ademas de oversampleada con imb_lear (usar preprocessing_1est.py) 
'''

'''
    Parametros
'''
balance_ratio = 1.0
#home = os.environ['HOME']
muestras_train = 0
muestras_val = 0
shape = (3,175,175) # grilla de 96x144 con 3 canales
#input_ten = Input(shape)
#umbral = 0.2
x_train_dir = "X_train.npy"
x_val_dir   = "X_val.npy"
y_train_dir = "Y_train.npy"
y_val_dir   = "Y_val.npy"
#model_dir   = home + "/modelos/CerroObero/24horas/not_outliers" + "/epoca{epoch:02d}.hdf5"
cant_epocas = 25
tam_batch = 48 # intentar que sea multiplo de la cantidad de muestras

'''
    Definicion de metricas personalizadas para evaluar en cada epoca y Checkpoints.
'''

class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_targ = self.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        _val_balanced_acc = balanced_accuracy_score(val_targ, val_predict)
        _val_zero_one_loss = zero_one_loss(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print("| val_zero_one_loss: {:.3f} | val_f1: {:.3f} | val_precision: {:.3f} | val_recall: {:.3f} | val_balanced_acc: {:.3f}".format(_val_zero_one_loss, _val_f1, _val_precision, _val_recall, _val_balanced_acc))
        return

##metrics = Metrics()
##checkpoint = ModelCheckpoint(model_dir, monitor='val_loss', verbose=1, save_best_only=False)
##callbacks_list = [checkpoint, metrics]

'''
    Definicion del modelo
'''

def get_vgg16():
    # we initialize the model
    model = Sequential()

    # Conv Block 1
    model.add(BatchNormalization(axis=3, input_shape=shape))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Conv Block 2
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Conv Block 3
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    #model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Conv Block 4
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    #model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Conv Block 5
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    #model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # FC layers
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    #adam = Adam(lr=0.001)
    sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    #print(model.summary())
    return model

'''
    Creacion del modelo
'''
model = get_vgg16()

'''
    Carga de datos
'''
x_train = np.load(x_train_dir)
x_val   = np.load(x_val_dir)
y_train = np.load(y_train_dir)
y_val   = np.load(y_val_dir)

'''
    Entrenamiento
'''
model.fit(x_train, y_train, batch_size=tam_batch, epochs=cant_epocas, verbose=1, validation_data=(x_val, y_val))

Epoch 1/25


ValueError: in user code:

    /home/Cavanagh_jf/miniconda3/envs/tesis/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/Cavanagh_jf/miniconda3/envs/tesis/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/Cavanagh_jf/miniconda3/envs/tesis/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/Cavanagh_jf/miniconda3/envs/tesis/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/Cavanagh_jf/miniconda3/envs/tesis/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(

In [ ]:
backend.set_image_data_format('channels_first')

In [41]:
# Loading the Base Model
from tensorflow.keras.applications.vgg16 import VGG16
shape = (3,175,175) # grilla de 96x144 con 3 canales
input_tenn = Input(shape=(shape))
base_model = VGG16(input_shape = (3,175,175), # Shape of our images
                   input_tensor =  input_tenn,
                   include_top = False, # Leave out the last fully connected layer
                   weights = None)

base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 3, 175, 175)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 175, 175)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 175, 175)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 87, 87)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 87, 87)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 87, 87)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 43, 43)       0     

In [42]:
# Como no tenemos q mejorar todas las capas, las hacemos non_trainable
for layer in base_model.layers:
    layer.trainable = False

In [43]:
# Compile and Fit
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x) #por que 1 y no 2? si son 2 clases

model = tensorflow.keras.models.Model(base_model.input, x)

model.compile(optimizer = tensorflow.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 3, 175, 175)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 175, 175)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 175, 175)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 87, 87)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 87, 87)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 87, 87)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 43, 43)       0   

In [49]:
X_train_dir = "X_train.npy"
X_val_dir   = "X_val.npy"
Y_train_dir = "Y_train.npy"
Y_val_dir   = "Y_val.npy"

X_train = np.load(X_train_dir)
X_val   = np.load(X_val_dir)
Y_train = np.load(Y_train_dir)
Y_val   = np.load(Y_val_dir)

print(Y_train.shape)
print(Y_val.shape)

#X_train =  np.reshape(X_train, (X_train.shape[0],175,175,3))
#X_val   =  np.reshape(X_val, (X_val.shape[0],175,175,3))
Y_train = Y_train.reshape(Y_train.shape[0], 30625)
Y_val   = Y_val.reshape(Y_val.shape[0], 30625)

print(Y_train.shape)
print(Y_val.shape)

vgghist = model.fit(X_train, Y_train, validation_data = (X_val,Y_val), steps_per_epoch = 100, epochs = 5)

(1904, 175, 175)
(408, 175, 175)
(1904, 30625)
(408, 30625)
Epoch 1/5


ValueError: in user code:

    /home/Cavanagh_jf/miniconda3/envs/tesis/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/Cavanagh_jf/miniconda3/envs/tesis/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/Cavanagh_jf/miniconda3/envs/tesis/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/Cavanagh_jf/miniconda3/envs/tesis/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/Cavanagh_jf/miniconda3/envs/tesis/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(

In [48]:
Y_train = np.load(Y_train_dir)
Y_train = Y_train.reshape(Y_train.shape[0], 30625)
print(Y_train.shape)

(1904, 30625)
